In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import time
import glob
import csv
import operator

## Calculates frequency of units grouped by properties(but only for interesting properties)

In [2]:
# Characteristic_Names_Map_Match contains mapping of columns which are synonyms of each other
df = pd.read_csv("Characteristic_Names_Map_Match.csv", encoding='UTF-8')
# Create a map for original propertyName to normalized property Name
dict = {}
for (idx,  row) in df.iterrows():
    property1 = row.loc['CharacteristicName_1'].strip()
    property2 = row.loc['CharacteristicName_2'].strip()
    dict[property1] = property2;
    dict[property2] = property2;
# This file has a column which contains names of properties we are interested in.
df = pd.read_csv("Interested_Variables.csv", encoding='UTF-8')
for (idx,  row) in df.iterrows():
    property1 = row.loc['CharacteristicName'].strip()
    if(property1 not in dict):
        dict[property1] = property1;

In [4]:
df = pd.read_csv("group_histogram_v2.csv", encoding='UTF-8')
f = csv.writer(open("interested_property_to_unit.csv", "w"))
f.writerow(['CharacteristicName', 'ResultMeasure/MeasureUnitCode' , "Frequency"])
merged_readings = {};
#Merges freq stats of different properties as now two different properties might point to same property
for (idx, row) in df.iterrows():
    propertyName = row.loc['CharacteristicName'].strip()
    if(propertyName not in dict):
        continue;
    propertyName = dict[propertyName];
    unit = str(row.loc['ResultMeasure/MeasureUnitCode']).strip()
    freq = row.loc['Frequency']
    if(propertyName not in merged_readings):
        merged_readings[propertyName] = {}
    if(unit not in merged_readings[propertyName]):
        merged_readings[propertyName][unit] = 0
    merged_readings[propertyName][unit] = merged_readings[propertyName][unit] + freq;
#Prints out new normalized and merged stats to a file
for propertyName in merged_readings:
    for unit in merged_readings[propertyName]:
        f.writerow([propertyName, unit, merged_readings[propertyName][unit]])